<a href="https://colab.research.google.com/github/Jaydeep-2000/Deep-Learning-with-Pytorch/blob/main/Build_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


In [15]:
'''
We define our neural network by subclassing nn.Module, and initialize the neural
 __init__. Every nn.Module subclass implements the operations on input data in 
 the forward method.
 '''
class Net(nn.Module):
   def __init__ (self):
     super(Net, self).__init__()
     self.flatten = nn.Flatten
     self.linear_relu_stack = nn.Sequential(
         nn.Linear(28*28, 512),
         nn.ReLU(),
         nn.Linear(512, 512),
         nn.ReLU(),
         nn.Linear(512, 10),
     )

   def forward(self, x):
      x = self.flatten(x)
      logits = self.linear_relu_stack(x)
      return logits




In [16]:
model = Net().to(device)
print(model)

Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
X = torch.rand(3, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

TypeError: ignored

**Model Layers**

In [19]:
'''
To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.
'''

input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [20]:
'''
We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).
'''
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [21]:
'''
The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.
'''
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [22]:
'''
Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity,
 helping neural networks learn a wide variety of phenomena.

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.
'''
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.2484, -0.8538, -0.4683,  0.0769,  0.1807,  0.0504,  0.2028, -0.0468,
          0.1639, -0.5241,  0.0323,  0.3538,  0.0337,  0.2866, -0.2241, -0.6194,
         -0.1735, -0.2633,  0.2982,  0.1025],
        [ 0.7418, -0.6277, -0.4733,  0.2182, -0.2370, -0.3953,  0.3420, -0.2882,
          0.5152, -0.2719,  0.0869,  0.0305, -0.0649, -0.1740,  0.0752, -0.5981,
         -0.3873, -0.5019,  0.3878,  0.0714],
        [ 0.4584, -0.6330, -0.2388, -0.3927,  0.1064, -0.0271,  0.3208, -0.3683,
          0.6431, -0.4686, -0.4265,  0.1819,  0.1512,  0.2492,  0.1738, -0.5074,
          0.0592, -0.4275,  0.3046, -0.1400]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.2484, 0.0000, 0.0000, 0.0769, 0.1807, 0.0504, 0.2028, 0.0000, 0.1639,
         0.0000, 0.0323, 0.3538, 0.0337, 0.2866, 0.0000, 0.0000, 0.0000, 0.0000,
         0.2982, 0.1025],
        [0.7418, 0.0000, 0.0000, 0.2182, 0.0000, 0.0000, 0.3420, 0.0000, 0.5152,
         0.0000, 0.0869, 0.0305, 0.0000, 0.0000, 0.07

In [23]:
'''
nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put 
together a quick network like seq_modules.
'''
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [24]:
'''
The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. 
The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.
'''
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [25]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0296,  0.0098,  0.0332,  ..., -0.0280,  0.0094,  0.0354],
        [-0.0255, -0.0025,  0.0205,  ...,  0.0223,  0.0256,  0.0159]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0322, -0.0256], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0165,  0.0360, -0.0155,  ..., -0.0141,  0.0117,  0.0420],
        [ 0.0426,  0.0058, -0.0218,  ..., -0.0245,  0.0170,  0.0322]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([-0.0254, -0